In [19]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.style.use('seaborn-notebook')
%matplotlib inline
import src.scrubbing_develop as scrubbing

In [2]:
# import yelp prices pickle file

In [3]:
df = pd.read_pickle('data/yelp_prices.pkl')

In [5]:
#df.head(15)

In [6]:
prices_lst = list(df.iloc[:,0])

In [7]:
len(prices_lst)

8529

In [8]:
# extract restaurant names first

In [9]:
rnames = []
for line in prices_lst:
    #print(line[1:6])
    if line[1:6] == 'span>':
        name = line.split('<span>')
        name2 = name[1].split('</span>')
        rnames.append(name2[0])

In [10]:
len(rnames)

4431

In [11]:
r_prices = []
for line in prices_lst:
    if line[1:7] == 'span c':
        name = line.split('">')
        name2 = name[1].split('</span>')
        r_prices.append(name2[0])

In [12]:
prices_lst = list(map(len, r_prices))

In [13]:
# let's join these two lists together

In [14]:
yelp_prices = dict(zip(rnames, prices_lst))

In [15]:
#yelp_prices

In [16]:
len(yelp_prices)

3006

In [17]:
# match restaurant names and input to price column. This process will go after step remove_rows_zero_violation

In [22]:
df2 = pd.read_csv('data/Restaurant_Scores_-_LIVES_Standard.csv')
df_last = scrubbing.scrub_all(df2, 'a')

/home/soong/galv/capstone/SFeaters/src/scrubbing_develop.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_viol['short_violation_id'] = L_vid
/home/soong/galv/capstone/SFeaters/src/scrubbing_develop.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['inspect_date'] = pd.to_datetime(df.inspection_date)
/home/soong/galv/capstone/SFeaters/src/scrubbing_develop.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [23]:
df_last.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,...,94129,94130,94131,94132,94133,94134,94143,94158,95105,p7_36
0,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,-122.420455,"(37.789784, -122.420455)",NaN,...,0,0,0,0,0,0,0,0,0,16
1,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,-122.419066,"(37.759174, -122.419066)",1.415583e+10,...,0,0,0,0,0,0,0,0,0,13
2,79782,Deli 23,2449 23rd St,San Francisco,CA,94110,37.754343,-122.403989,NaN,NaN,...,0,0,0,0,0,0,0,0,0,10
3,73840,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,6
4,76437,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,37.795305,-122.406613,NaN,NaN,...,0,0,0,0,0,0,0,0,0,12


In [24]:
for bname in df_last['business_name']:
    pass
    #print(bname)

In [25]:
for yname, price in yelp_prices.items():
    for bname in df_last['business_name']:
        yname_c = yname.strip().lower()
        bname_c = bname.strip().lower()
        if yname_c[:10] == bname_c[:10]:
            idx = df_last.loc[df_last['business_name'] == bname].index
            #print(idx)
            df_last.loc[idx, 'price'] = price

In [153]:
# let's give un-priced restaurants the average value.

In [17]:
mask = df_last['price'].isnull()

In [18]:
avg_price = df_last.loc[~mask, 'price'].sum()/len(df_last.loc[~mask, 'price'])

In [19]:
df_last.loc[mask,'price'] = avg_price

In [71]:
# Do not run this unless you want the new data with prices
#df_last.to_pickle('data/sf_inspection_w_prices.pkl')

In [20]:
avg_price

1.7681924882629108